<a href="https://colab.research.google.com/github/creatino/creatino.github.io/blob/master/Stock_Portfolio_v1_02022025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import yfinance as yf
import datetime
import calendar
import os

# Define the path to the Excel file containing stock data
file_path = '../content/gdrive/My Drive/Colab_Notebooks/StockAnalysis/Top_Stocks.xlsx'

# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(file_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    exit()  # Stop execution if the file is not found

# Extract the stock symbols from the DataFrame
stock_symbols = df['Symbol'].tolist()

# Print the extracted stock symbols
print("Extracted Stock Symbols:", stock_symbols)


# Define a function to calculate stock growth data
def calculate_percentage_growth(stock_symbols):
    # Define a dictionary containing different time periods
    periods = {
        '5 Days': 5,
        '10 Days': 10,
        '1 Month': 30,
        '3 Months': 90,
        '6 Months': 180
    }

    # Initialize a list to store growth data
    growth_data = []

    # Iterate over each stock symbol
    for symbol in stock_symbols:
        # Try downloading historical stock data using yfinance
        try:
            data = yf.download(symbol, period="2y")['Close']
            # Get the current date
            today = datetime.date.today()

            # Initialize an empty dictionary to store growth data for the current symbol
            growth = {}

            # Iterate over each time period
            for period_name, period_days in periods.items():
                # Calculate the past date based on the period
                past_date = today - datetime.timedelta(days=period_days)

                # Adjust the past date to the previous weekday if it falls on a weekend
                while past_date.weekday() in (calendar.SATURDAY, calendar.SUNDAY):
                    past_date -= datetime.timedelta(days=1)

                # Filter the data to include only prices from the past date up to today
                data_filtered = data[data.index.date >= past_date]

                # Check if there are enough data points to calculate growth
                if len(data_filtered) >= 2:
                    # Calculate the percentage change between the current and past prices
                    current_price = data_filtered.iloc[-1]
                    past_price = data_filtered.iloc[0]
                    percentage_change = ((current_price - past_price) / past_price) * 100

                    # Store the percentage change in the growth dictionary
                    growth[period_name] = percentage_change
                else:
                    # If there aren't enough data points, set the growth percentage to None
                    growth[period_name] = None

            # Append the growth dictionary for the current symbol to the growth_data list
            growth_data.append(growth)

        # Catch any errors that occur during the data download or calculation
        except Exception as e:
            # Print an error message indicating the symbol and the error that occurred
            print(f"Error calculating growth for {symbol}: {e}")
            growth_data.append({period_name: None for period_name in periods})  # Append None values for all periods

    # Create a DataFrame from the growth data using the stock symbols as the index
    df_growth = pd.DataFrame(growth_data, index=stock_symbols)

    # Return the DataFrame containing the growth data
    return df_growth


# Call the function to calculate and assign the result to df_growth
df_growth = calculate_percentage_growth(stock_symbols)

# Ensure the index is unique
df_growth.index = df_growth.index.astype(str) + '_' + df_growth.groupby(level=0).cumcount().astype(str)

# Create an empty dictionary to store the stock data
stock_data = {}

# Iterate over each stock symbol
for symbol in stock_symbols:
    # Try downloading stock data using yfinance
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info

        # Extract relevant data points from the info dictionary
        stock_data[symbol] = {
            "Current Stock Price": info.get("currentPrice"),
            "Market Capitalization": info.get("marketCap"),
            "Trailing P/E": info.get("trailingPE"),
            "Forward P/E": info.get("forwardPE"),
            "Price-to-Book Ratio": info.get("priceToBook"),
            "Price-to-Sales Ratio": info.get("priceToSalesTrailing12Months"),
            "52 Week High": info.get("fiftyTwoWeekHigh"),
            "52 Week Low": info.get("fiftyTwoWeekLow"),
            "Dividend Yield": info.get("dividendYield"),

            # Profitability
            "ROA": info.get("returnOnAssets"),
            "Gross Profit Margin": info.get("grossMargins"),
            "EPS": info.get("trailingEps"),
            "Profit Margin": info.get("profitMargins"),
            "ROE": info.get("returnOnEquity"),

            # Financial Health
            "Debt-to-Equity Ratio": info.get("debtToEquity"),
            "Current Ratio": info.get("currentRatio"),
            "Interest Coverage Ratio": info.get("interestCoverage"),
            "Debt-to-Asset Ratio": info.get("totalDebt") / info.get("totalAssets") if info.get("totalDebt") and info.get(
                "totalAssets") else None,
            "Cash Flow": info.get("operatingCashflow"),

            # Valuation
            "PEG Ratio": info.get("pegRatio"),

            # Growth
            "Revenue Growth": info.get("revenueGrowth"),
            "Earnings Growth Rate": info.get("earningsGrowth"),

            # Analyst Ratings
            "Analyst Recommendation": info.get("recommendationKey"),
            "Number of Analyst Opinions": info.get("numberOfAnalystOpinions"),
            "Target Low Price": info.get("targetLowPrice"),
            "Target High Price": info.get("targetHighPrice"),
            "Target Mean Price": info.get("targetMeanPrice"),
            "Target Median Price": info.get("targetMedianPrice"),

            # Short Interest
            "Short Interest": info.get("sharesShort"),
            "Short Ratio": info.get("shortRatio"),
            "Short Percent of Float": info.get("shortPercentOfFloat"),

            # Institutional Ownership
            "Held Percent Institutions": info.get("heldPercentInstitutions"),
            "Held Percent Insiders": info.get("heldPercentInsiders"),

            # Trading Volume
            "Average Volume": info.get("averageVolume"),
            "Volume": info.get("volume"),
        }
    # Catch any errors that occur during the data download
    except Exception as e:
        # Print an error message indicating the symbol and the error that occurred
        print(f"Error fetching data for {symbol}: {e}")
        # Set the data for the symbol to 'Data not available' if there was an error
        stock_data[symbol] = "Data not available"

# Convert the stock data dictionary to a DataFrame
stock_df = pd.DataFrame(stock_data).transpose()

# Reset the index of the DataFrame to have 'Symbol' as a column
stock_df.reset_index(inplace=True)

# Rename the index column to 'Symbol' for clarity
stock_df.rename(columns={'index': 'Symbol'}, inplace=True)

# Define the output path to the output files
output_file_path = '../content/gdrive/My Drive/Colab_Notebooks/StockAnalysis/'

# Construct the output file paths
stock_fundamentals_path = os.path.join(os.path.dirname(output_file_path), 'stock_fundamentals.xlsx')
stock_growth_path = os.path.join(os.path.dirname(output_file_path), 'stock_growth.xlsx')

# Write the DataFrames to separate Excel files
stock_df.to_excel(stock_fundamentals_path, index=False)
df_growth.to_excel(stock_growth_path, index=False)

# Print success messages
print(f"Stock fundamental data saved to: {stock_fundamentals_path}")
print(f"Stock growth data saved to: {stock_growth_path}")

Extracted Stock Symbols: ['SEZL', 'APP', 'RDDT', 'MSTR', 'PLTR', 'CVNA', 'LB', 'ALAB', 'GGAL', 'VST', 'PRAX', 'CRDO', 'BMA', 'IONQ', 'CLS', 'TLN', 'FTAI', 'EAT', 'AGX', 'HOOD', 'USLM', 'TECX', 'SMTC', 'VERA', 'TARS', 'NVDA', 'SFM', 'LOAR', 'CAVA', 'SE', 'IESC', 'NTRA', 'POWL', 'GEV', 'NGVC', 'YPF', 'DXPE', 'QFIN', 'VITL']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Stock fundamental data saved to: ../content/gdrive/My Drive/Colab_Notebooks/StockAnalysis/stock_fundamentals.xlsx
Stock growth data saved to: ../content/gdrive/My Drive/Colab_Notebooks/StockAnalysis/stock_growth.xlsx


In [5]:
pip install pandas yfinance openpyxl GoogleNews textblob nltk

In [ ]:
import pandas as pd
import yfinance as yf
from GoogleNews import GoogleNews
from textblob import TextBlob
import os
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download NLTK resources (if not already downloaded)
nltk.download('vader_lexicon', quiet=True)
nltk.download('punkt', quiet=True)

def get_news_and_analyze_sentiment(stock_symbols):
    """
    Fetches news articles, analyst opinions (if available), performs sentiment analysis,
    and returns a DataFrame with the results.

    Args:
        stock_symbols (list): A list of stock symbols.

    Returns:
        pandas.DataFrame: A DataFrame with sentiment analysis results.
    """

    sentiment_data = []

    for symbol in stock_symbols:
        print(f"Processing {symbol}...")

        # Fetch News
        googlenews = GoogleNews()
        googlenews.search(symbol)
        news_results = googlenews.results(sort=True)

        # Fetch Analyst Opinion (using yfinance as a proxy - limited data)
        ticker = yf.Ticker(symbol)
        try:
            analyst_recommendation = ticker.info.get("recommendationKey")
        except:
            analyst_recommendation = "Not Available"

        # Sentiment Analysis
        news_sentiment_scores = []

        print(f"  News clips for {symbol}:")  # Print header for news clips

        for article in news_results:
            try:
                analysis = TextBlob(article['desc'])
                news_sentiment_scores.append(analysis.sentiment.polarity)
                print(f"    - {article['title']}: {article['desc'][:100]}...")  # Print title and snippet of description
            except Exception as e:
                print(f"    - Error analyzing sentiment for article: {article['title']}, Error: {e}")

        # Calculate Average Sentiment
        avg_news_sentiment = sum(news_sentiment_scores) / len(news_sentiment_scores) if news_sentiment_scores else 0

        # Classify Sentiment
        news_sentiment_label = "Positive" if avg_news_sentiment > 0.1 else "Negative" if avg_news_sentiment < -0.1 else "Neutral"

        #using NLTK for sentiment analysis
        analyzer = SentimentIntensityAnalyzer()

        # Perform sentiment analysis on news articles
        news_sentiment_nltk = []

        for article in news_results:
            try:
                scores = analyzer.polarity_scores(article['desc'])
                news_sentiment_nltk.append(scores)

            except Exception as e:
                print(f"Error analyzing sentiment for article: {article['title']}, Error: {e}")

        # Calculate average sentiment scores
        avg_news_compound = sum(score['compound'] for score in news_sentiment_nltk) / len(news_sentiment_nltk) if news_sentiment_nltk else 0

        # Classify sentiment based on compound score
        if avg_news_compound >= 0.05:
            news_sentiment_label_nltk = "Positive"
        elif avg_news_compound <= -0.05:
            news_sentiment_label_nltk = "Negative"
        else:
            news_sentiment_label_nltk = "Neutral"

        sentiment_data.append({
            "Symbol": symbol,
            "Analyst Recommendation": analyst_recommendation,
            "Average News Sentiment": avg_news_sentiment,
            "News Sentiment Label": news_sentiment_label,
            "Average News Sentiment NLTK": avg_news_compound,
            "News Sentiment Label NLTK" : news_sentiment_label_nltk
        })

        googlenews.clear()

    return pd.DataFrame(sentiment_data)

def save_sentiment_to_excel(sentiment_df, output_path):
    """
    Saves the sentiment analysis DataFrame to an Excel file.

    Args:
        sentiment_df (pandas.DataFrame): The DataFrame with sentiment data.
        output_path (str): The path to save the Excel file.
    """
    sentiment_file_path = os.path.join(output_path, 'sentiment.xlsx')
    sentiment_df.to_excel(sentiment_file_path, index=False)
    print(f"Sentiment analysis data saved to: {sentiment_file_path}")

def main():
    """
    Main function to perform sentiment analysis on stocks.
    """
    # Define the path to the Excel file containing stock data
    file_path = '../content/gdrive/My Drive/Colab_Notebooks/StockAnalysis/Top_Stocks.xlsx'

    # Read the Excel file into a pandas DataFrame
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        exit()  # Stop execution if the file is not found

    # Extract the stock symbols from the DataFrame
    stock_symbols = df['Symbol'].tolist()

    # Define the output path for sentiment analysis results
    output_file_path = '../content/gdrive/My Drive/Colab_Notebooks/StockAnalysis/'

    # --- Sentiment Analysis ---
    sentiment_df = get_news_and_analyze_sentiment(stock_symbols)
    save_sentiment_to_excel(sentiment_df, output_file_path)

if __name__ == "__main__":
    main()

The code below is a copy but now prints the hyperlinks to the source of the news articles

In [6]:
import pandas as pd
import yfinance as yf
from GoogleNews import GoogleNews
from textblob import TextBlob
import os
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download NLTK resources (if not already downloaded)
nltk.download('vader_lexicon', quiet=True)
nltk.download('punkt', quiet=True)

# Mount Google Drive (if running in Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/My Drive/Colab_Notebooks/StockAnalysis/'  # Base path for Google Drive
except:
    base_path = ''  # Local execution, set base path accordingly


def get_news_and_analyze_sentiment(stock_symbols):
    """
    Fetches news articles, analyst opinions (if available), performs sentiment analysis,
    and returns a DataFrame with the results.

    Args:
        stock_symbols (list): A list of stock symbols.

    Returns:
        pandas.DataFrame: A DataFrame with sentiment analysis results.
    """

    sentiment_data = []

    for symbol in stock_symbols:
        print(f"Processing {symbol}...")

        # Fetch News
        googlenews = GoogleNews()
        googlenews.search(symbol)
        news_results = googlenews.results(sort=True)

        # Fetch Analyst Opinion (using yfinance as a proxy - limited data)
        ticker = yf.Ticker(symbol)
        try:
            analyst_recommendation = ticker.info.get("recommendationKey")
        except:
            analyst_recommendation = "Not Available"

        # Sentiment Analysis and News Clip Printing
        news_sentiment_scores = []

        print(f"  News clips for {symbol}:")

        for article in news_results:
            try:
                analysis = TextBlob(article['desc'])
                news_sentiment_scores.append(analysis.sentiment.polarity)
                # Print title, snippet of description, and link
                print(f"    - {article['title']}: {article['desc'][:100]}... Link: {article['link']}")
            except Exception as e:
                print(f"    - Error analyzing sentiment for article: {article['title']}, Error: {e}")

        # Calculate Average Sentiment
        avg_news_sentiment = sum(news_sentiment_scores) / len(news_sentiment_scores) if news_sentiment_scores else 0

        # Classify Sentiment
        news_sentiment_label = "Positive" if avg_news_sentiment > 0.1 else "Negative" if avg_news_sentiment < -0.1 else "Neutral"

        #using NLTK for sentiment analysis
        analyzer = SentimentIntensityAnalyzer()

        # Perform sentiment analysis on news articles
        news_sentiment_nltk = []

        for article in news_results:
            try:
                scores = analyzer.polarity_scores(article['desc'])
                news_sentiment_nltk.append(scores)

            except Exception as e:
                print(f"Error analyzing sentiment for article: {article['title']}, Error: {e}")

        # Calculate average sentiment scores
        avg_news_compound = sum(score['compound'] for score in news_sentiment_nltk) / len(news_sentiment_nltk) if news_sentiment_nltk else 0

        # Classify sentiment based on compound score
        if avg_news_compound >= 0.05:
            news_sentiment_label_nltk = "Positive"
        elif avg_news_compound <= -0.05:
            news_sentiment_label_nltk = "Negative"
        else:
            news_sentiment_label_nltk = "Neutral"

        sentiment_data.append({
            "Symbol": symbol,
            "Analyst Recommendation": analyst_recommendation,
            "Average News Sentiment": avg_news_sentiment,
            "News Sentiment Label": news_sentiment_label,
            "Average News Sentiment NLTK": avg_news_compound,
            "News Sentiment Label NLTK": news_sentiment_label_nltk
        })

        googlenews.clear()

    return pd.DataFrame(sentiment_data)

def save_sentiment_to_excel(sentiment_df, output_path):
    """
    Saves the sentiment analysis DataFrame to an Excel file.
    """
    sentiment_file_path = os.path.join(output_path, 'sentiment.xlsx')
    try:
        sentiment_df.to_excel(sentiment_file_path, index=False)
        print(f"Sentiment analysis data saved to: {sentiment_file_path}")
    except IOError as e:
        print(f"Error saving sentiment data to Excel: {e}")

def main():
    """
    Main function to perform sentiment analysis on stocks.
    """
    # Define the path to the Excel file containing stock data (relative to base_path)
    file_path = os.path.join(base_path, 'Top_Stocks.xlsx')

    # Read the Excel file
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        exit()

    # Extract stock symbols
    stock_symbols = df['Symbol'].tolist()

    # Define the output path for sentiment analysis results (same as base_path)
    output_file_path = base_path

    # --- Sentiment Analysis ---
    sentiment_df = get_news_and_analyze_sentiment(stock_symbols)
    save_sentiment_to_excel(sentiment_df, output_file_path)

if __name__ == "__main__":
    main()

Mounted at /content/drive
Processing SEZL...
HTTP Error 429: Too Many Requests
  News clips for SEZL:
Processing APP...
HTTP Error 429: Too Many Requests
  News clips for APP:
Processing RDDT...
HTTP Error 429: Too Many Requests
  News clips for RDDT:
Processing MSTR...
HTTP Error 429: Too Many Requests
  News clips for MSTR:
Processing PLTR...
HTTP Error 429: Too Many Requests
  News clips for PLTR:
Processing CVNA...
HTTP Error 429: Too Many Requests
  News clips for CVNA:
Processing LB...
HTTP Error 429: Too Many Requests
  News clips for LB:
Processing ALAB...
HTTP Error 429: Too Many Requests
  News clips for ALAB:
Processing GGAL...
HTTP Error 429: Too Many Requests
  News clips for GGAL:
Processing VST...
HTTP Error 429: Too Many Requests
  News clips for VST:
Processing PRAX...
HTTP Error 429: Too Many Requests
  News clips for PRAX:
Processing CRDO...
HTTP Error 429: Too Many Requests
  News clips for CRDO:
Processing BMA...
HTTP Error 429: Too Many Requests
  News clips for B